In [35]:
import json
from pathlib import Path 
import requests
import pandas as pd
from datetime import date, timedelta
import time

In [3]:
home = str(Path.home())
with open(home + "/bfg/demo/config.json", encoding = 'utf-8') as f:
    config = json.load(f)

In [ ]:
payload = {
    "encryptedPassword": False,
    "identifier": config['connection_details']['username'],
    "password": config['connection_details']['password'],
}
headers = {
    "Version": "2",
    "X-IG-API-KEY": config['connection_details']['api_key'],
    "Content-Type": "application/json; charset=UTF-8",
    "Accept": "application/json; charset=UTF-8",
          }
r = requests.post(config['connection_details']['base_url'] + "session", headers=headers, data=json.dumps(payload))
xst=r.headers["x-security-token"]
cst=r.headers["cst"]

In [38]:
def fetchData(epic, resolution, start):
    resource = f"prices/{epic}?resolution={resolution}&from={start}&to={start}"
    headers = {
        "X-SECURITY-TOKEN": xst,
        "CST": cst,
        "Version": "3",
        "X-IG-API-KEY": config['connection_details']['api_key'],
        "Content-Type": "application/json; charset=UTF-8",
        "Accept": "application/json; charset=UTF-8",
              }
    r = requests.get(config['connection_details']['base_url'] + resource, headers=headers,)
    if r.status_code != 200:
        print(r.text)
    else:
        return r.json()

In [ ]:
def makePandasRow(min1, min5):
    price1 = cmin1['prices'][0]
    price5 = min5['prices'][0]
    return {
        "utc_time": price1['snapshotTimeUTC'],
        "1min_bid_high": price1['highPrice']['bid'],
        "1min_ask_high": price1['highPrice']['ask'],
        "1min_bid_low": price1['lowPrice']['bid'],
        "1min_ask_low": price1['lowPrice']['ask'],
        "5min_bid_high": price5['highPrice']['bid'],
        "5min_ask_high": price5['highPrice']['ask'],
        "5min_bid_low": price5['lowPrice']['bid'],
        "5min_ask_low": price5['lowPrice']['ask'],
    }

IG has 40days of 1min data and 360days of 5min data
Rate limited to 30req/min and we do 2 requests each loop so need to sleep

In [42]:
forty_days_from_today = date.today()-timedelta(days=40)
for epic in config['epics']:
    rows = []
    non_trading_days = []    
    year_dates = pd.bdate_range(start=forty_days_from_today, end=date.today(), holidays=epic['non_trading_days'])
    for day in year_dates:
        time.sleep(4.5)
        min1 = fetchData(epic['epic'], "MINUTE", f"{str(day.date())}T09:00:00")
        min5 = fetchData(epic['epic'], "MINUTE_5", f"{str(day.date())}T09:00:00")
        row = makePandasRow(min1, min5)
        rows.append(row)
    df = pd.DataFrame(rows)
    df.set_index('utc_time', inplace=True)
    df.to_csv(f"opening_ranges_2022_{epic['epic']}.csv")